In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import os
if not os.path.exists('deepSORT_MOT17'):
  !git clone https://github.com/Ulyssesllc/deepSORT_MOT17.git
else:
  !git -C deepSORT_MOT17 pull --ff-only

Cloning into 'deepSORT_MOT17'...
remote: Enumerating objects: 11507, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 11507 (delta 4), reused 8 (delta 2), pack-reused 11494 (from 2)
Receiving objects: 100% (11507/11507), 1.86 GiB | 27.74 MiB/s, done.
Resolving deltas: 100% (36/36), done.


In [3]:
# Sync MOT17 directly from Drive into the repo to keep the repo slim
repo_root = '/content/deepSORT_MOT17'
drive_zip = '/content/drive/MyDrive/MOT17.zip'
drive_dir = '/content/drive/MyDrive/MOT17'

import os
os.makedirs(repo_root, exist_ok=True)

# Prefer syncing the folder if it exists (faster, incremental). Otherwise unzip the ZIP directly into the repo.
if os.path.exists(drive_dir):
  print('Syncing MOT17 folder from Drive ->', f'{repo_root}/MOT17')
  !rsync -a --delete "/content/drive/MyDrive/MOT17/" "/content/deepSORT_MOT17/MOT17/"
elif os.path.exists(drive_zip):
  print('Unzipping MOT17.zip from Drive into repo ->', repo_root)
  !unzip -q "/content/drive/MyDrive/MOT17.zip" -d "/content/deepSORT_MOT17"
else:
  raise SystemExit('MOT17 source not found on Drive. Provide MOT17/ folder or MOT17.zip in MyDrive.')

# Verify a few entries
print('Top-level sequences under repo MOT17/:')
!ls -1 "/content/deepSORT_MOT17/MOT17" | head -n 10

Unzipping MOT17.zip from Drive into repo -> /content/deepSORT_MOT17
Top-level sequences under repo MOT17/:
test
train


In [ ]:
import os

os.chdir('deepSORT_MOT17')
!pip install -r requirements.txt
# Install lap for motmetrics linear assignment
!pip install lap
# Ensure TF packages available on Colab for feature extraction
!pip install tensorflow tf-keras tf-slim
# Ensure ffmpeg is present for video conversion
!apt-get update -y && apt-get install -y ffmpeg



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.5/161.5 kB 5.2 MB/s eta 0:00:00
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 https://cli.github.com/packages stable InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,199 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubunt

In [5]:
# Ensure the MARS ReID model exists at expected path
import pathlib
model_dst = pathlib.Path('resources/networks/mars-small128.pb')
model_dst.parent.mkdir(parents=True, exist_ok=True)
# If you have it on Drive, copy it; else download from your own link
# Example (adjust the source path accordingly):
# !cp /content/drive/MyDrive/models/mars-small128.pb resources/networks/mars-small128.pb
print('Model exists:', model_dst.exists(), '->', model_dst)

Model exists: True -> resources/networks/mars-small128.pb


In [6]:
# Optional: validate after features are generated. Skip now to avoid failing before creation.
# !python validate_paths.py \
#   --sequence_dir MOT17/train/MOT17-04-FRCNN \
#   --detection_file detections/MOT17-04-FRCNN.npy
print('Skipping validation pre-generation. Will validate after features are created.')

Skipping validation pre-generation. Will validate after features are created.


In [ ]:
!python tools/generate_detections.py \
  --model resources/networks/mars-small128.pb \
  --mot_dir MOT17/train \
  --output_dir detections

Streaming output truncated to the last 5000 lines.
Frame 00611/00654
Frame 00612/00654
Frame 00613/00654
Frame 00614/00654
Frame 00615/00654
Frame 00616/00654
Frame 00617/00654
Frame 00618/00654
Frame 00619/00654
Frame 00620/00654
Frame 00621/00654
Frame 00622/00654
Frame 00623/00654
Frame 00624/00654
Frame 00625/00654
Frame 00626/00654
Frame 00627/00654
Frame 00628/00654
Frame 00629/00654
Frame 00630/00654
Frame 00631/00654
Frame 00632/00654
Frame 00633/00654
Frame 00634/00654
Frame 00635/00654
Frame 00636/00654
Frame 00637/00654
Frame 00638/00654
Frame 00639/00654
Frame 00640/00654
Frame 00641/00654
Frame 00642/00654
Frame 00643/00654
Frame 00644/00654
Frame 00645/00654
Frame 00646/00654
Frame 00647/00654
Frame 00648/00654
Frame 00649/00654
Frame 00650/00654
Frame 00651/00654
Frame 00652/00654
Frame 00653/00654
Frame 00654/00654
Processing MOT17-11-DPM
Frame 00001/00900
Frame 00002/00900
Frame 00003/00900
Frame 00004/00900
Frame 00005/00900
Frame 00006/00900
Frame 00007/00900
Frame 0

In [ ]:
# Also generate features for the test split (no gt there)
!python tools/generate_detections.py \
  --model resources/networks/mars-small128.pb \
  --mot_dir MOT17/test \
  --output_dir detections

Streaming output truncated to the last 5000 lines.
Frame 00582/00625
Frame 00583/00625
Frame 00584/00625
Frame 00585/00625
Frame 00586/00625
Frame 00587/00625
Frame 00588/00625
Frame 00589/00625
Frame 00590/00625
Frame 00591/00625
Frame 00592/00625
Frame 00593/00625
Frame 00594/00625
Frame 00595/00625
Frame 00596/00625
Frame 00597/00625
Frame 00598/00625
Frame 00599/00625
Frame 00600/00625
Frame 00601/00625
Frame 00602/00625
Frame 00603/00625
Frame 00604/00625
Frame 00605/00625
Frame 00606/00625
Frame 00607/00625
Frame 00608/00625
Frame 00609/00625
Frame 00610/00625
Frame 00611/00625
Frame 00612/00625
Frame 00613/00625
Frame 00614/00625
Frame 00615/00625
Frame 00616/00625
Frame 00617/00625
Frame 00618/00625
Frame 00619/00625
Frame 00620/00625
Frame 00621/00625
Frame 00622/00625
Frame 00623/00625
Frame 00624/00625
Frame 00625/00625
Processing MOT17-12-DPM
Frame 00001/00900
Frame 00002/00900
Frame 00003/00900
Frame 00004/00900
Frame 00005/00900
Frame 00006/00900
Frame 00007/00900
Frame 0

In [9]:
# Now that features exist, validate one sequence path and its .npy
!python validate_paths.py \
  --sequence_dir MOT17/train/MOT17-04-FRCNN \
  --detection_file detections/MOT17-04-FRCNN.npy

[OK ] sequence_dir: MOT17/train/MOT17-04-FRCNN
  - img1: OK
  - det/det.txt: OK
[OK ] detection_file: detections/MOT17-04-FRCNN.npy

Mọi thứ sẵn sàng để chạy deep_sort_app.py


In [10]:
!python evaluate_motchallenge.py \
  --mot_dir MOT17/train \
  --detection_dir detections \
  --output_dir results \
  --min_confidence 0.3 \
  --nms_max_overlap 1.0 \
  --max_cosine_distance 0.2 \
  --nn_budget 100

Streaming output truncated to the last 5000 lines.
Processing frame 00612
Processing frame 00613
Processing frame 00614
Processing frame 00615
Processing frame 00616
Processing frame 00617
Processing frame 00618
Processing frame 00619
Processing frame 00620
Processing frame 00621
Processing frame 00622
Processing frame 00623
Processing frame 00624
Processing frame 00625
Processing frame 00626
Processing frame 00627
Processing frame 00628
Processing frame 00629
Processing frame 00630
Processing frame 00631
Processing frame 00632
Processing frame 00633
Processing frame 00634
Processing frame 00635
Processing frame 00636
Processing frame 00637
Processing frame 00638
Processing frame 00639
Processing frame 00640
Processing frame 00641
Processing frame 00642
Processing frame 00643
Processing frame 00644
Processing frame 00645
Processing frame 00646
Processing frame 00647
Processing frame 00648
Processing frame 00649
Processing frame 00650
Processing frame 00651
Processing frame 00652
Proces

In [11]:
# (Optional) Run tracker on test split as well (no GT there)
!python evaluate_motchallenge.py \
  --mot_dir MOT17/test \
  --detection_dir detections \
  --output_dir results \
  --min_confidence 0.3 \
  --nms_max_overlap 1.0 \
  --max_cosine_distance 0.2 \
  --nn_budget 100

Streaming output truncated to the last 5000 lines.
Processing frame 00583
Processing frame 00584
Processing frame 00585
Processing frame 00586
Processing frame 00587
Processing frame 00588
Processing frame 00589
Processing frame 00590
Processing frame 00591
Processing frame 00592
Processing frame 00593
Processing frame 00594
Processing frame 00595
Processing frame 00596
Processing frame 00597
Processing frame 00598
Processing frame 00599
Processing frame 00600
Processing frame 00601
Processing frame 00602
Processing frame 00603
Processing frame 00604
Processing frame 00605
Processing frame 00606
Processing frame 00607
Processing frame 00608
Processing frame 00609
Processing frame 00610
Processing frame 00611
Processing frame 00612
Processing frame 00613
Processing frame 00614
Processing frame 00615
Processing frame 00616
Processing frame 00617
Processing frame 00618
Processing frame 00619
Processing frame 00620
Processing frame 00621
Processing frame 00622
Processing frame 00623
Proces

In [12]:
# Evaluate metrics on train split only (test split has no GT)
!python evaluate_metrics.py \
  --mot_dir MOT17/train \
  --result_dir results \
  --output_csv mot17_train_metrics.csv \
  --min_iou 0.5

Đánh giá MOT17-02-DPM ...
Traceback (most recent call last):
  File "/content/deepSORT_MOT17/evaluate_metrics.py", line 159, in <module>
    main()
  File "/content/deepSORT_MOT17/evaluate_metrics.py", line 148, in main
    summary, str_summary = evaluate_split(
                           ^^^^^^^^^^^^^^^
  File "/content/deepSORT_MOT17/evaluate_metrics.py", line 108, in evaluate_split
    acc.update(gt_ids, res_ids, dists)
  File "/usr/local/lib/python3.12/dist-packages/motmetrics/mot.py", line 252, in update
    rids, cids = linear_sum_assignment(dists)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/motmetrics/lap.py", line 73, in linear_sum_assignment
    rids, cids = solver(costs)
                 ^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/motmetrics/lap.py", line 288, in lsa_solve_lapjv
    from lap import lapjv
ModuleNotFoundError: No module named 'lap'


In [13]:
!python deep_sort_app.py \
  --sequence_dir MOT17/train/MOT17-04-FRCNN \
  --detection_file detections/MOT17-04-FRCNN.npy \
  --output_file results/MOT17-04-FRCNN.txt \
  --min_confidence 0.3 \
  --nms_max_overlap 1.0 \
  --max_cosine_distance 0.2 \
  --nn_budget 100 \
  --display False

Processing frame 00001
Processing frame 00002
Processing frame 00003
Processing frame 00004
Processing frame 00005
Processing frame 00006
Processing frame 00007
Processing frame 00008
Processing frame 00009
Processing frame 00010
Processing frame 00011
Processing frame 00012
Processing frame 00013
Processing frame 00014
Processing frame 00015
Processing frame 00016
Processing frame 00017
Processing frame 00018
Processing frame 00019
Processing frame 00020
Processing frame 00021
Processing frame 00022
Processing frame 00023
Processing frame 00024
Processing frame 00025
Processing frame 00026
Processing frame 00027
Processing frame 00028
Processing frame 00029
Processing frame 00030
Processing frame 00031
Processing frame 00032
Processing frame 00033
Processing frame 00034
Processing frame 00035
Processing frame 00036
Processing frame 00037
Processing frame 00038
Processing frame 00039
Processing frame 00040
Processing frame 00041
Processing frame 00042
Processing frame 00043
Processing 

In [ ]:
# Visualize results: on Colab, this will render frames to a window-less viewer; use video export + inline preview below for display
!python show_results.py \
  --sequence_dir MOT17/train/MOT17-04-FRCNN \
  --result_file results/MOT17-04-FRCNN.txt \
  --detection_file detections/MOT17-04-FRCNN.npy \
  --update_ms 20

Frame idx 1
qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.12/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



In [15]:
!python generate_videos.py \
  --mot_dir MOT17/train \
  --result_dir results \
  --detection_dir detections \
  --output_dir videos \
  --update_ms 20 \
  --convert_h264

Saving MOT17-09-SDP.txt to videos/MOT17-09-SDP.avi.
Frame idx 1
qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.12/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



In [16]:
# Inline preview the generated video on Colab
import os
from IPython.display import HTML
from base64 import b64encode
video_path = 'videos/MOT17-04-FRCNN.mp4'
if not os.path.exists(video_path):
  # Fallback to a likely name; list videos
  import glob
  vids = glob.glob('videos/*.mp4')
  print('Videos found:', vids)
  if vids:
    video_path = vids[0]
with open(video_path,'rb') as f:
  mp4 = f.read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=800 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

Videos found: []


FileNotFoundError: [Errno 2] No such file or directory: 'videos/MOT17-04-FRCNN.mp4'